In [3]:
import os
import shutil
import pandas as pd

# Define the column names with 30-minute intervals
columns = ['Dates', 'Station', 'Unit'] + [f"{hour:02d}{minute:02d}" for hour in range(24) for minute in (0, 30)]

In [ ]:
# Directory containing the Excel files
directory = r"Total Power Per Month File Path"
destination_folder = r"Time Series Total Power Generation Target Path"
year = '2019'

In [23]:
# List to hold dataframes
df_list = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        filepath = os.path.join(directory, filename)
        # Read the Excel file, skipping the header row initially
        df = pd.read_excel(filepath, header=None)
        # Assign column names
        df.columns = columns
        # Append the dataframe to the list
        df_list.append(df)

# Concatenate all dataframes
combined_df = pd.concat(df_list, ignore_index=True)

# Ensure the 'Dates' column is parsed as datetime
combined_df['Dates'] = pd.to_datetime(combined_df['Dates'])

# Sort the DataFrame by the 'Dates' column
combined_df.sort_values(by='Dates', inplace=True)

# Rename 'Dates' column to 'Day'
combined_df.rename(columns={'Dates': 'Day'}, inplace=True)

# Reshape the dataframe to long format
df_long = pd.melt(combined_df, id_vars=['Day', 'Station', 'Unit'], var_name='Time', value_name='Power Generation (MW)')

# Summarize the data based on daily frequency
daily_time_series = df_long.groupby('Day').sum().reset_index()

# Convert the 'Day' column to DD/MM/YYYY format
daily_time_series['Day'] = daily_time_series['Day'].dt.strftime('%Y-%m-%d')
daily_time_series

# Drop 'Station', 'Unit', and 'Time' columns as they are now redundant
daily_time_series.drop(columns=['Station', 'Unit', 'Time'], inplace=True) # For daily basis, drop Month column, but if monthly drop Day column
daily_time_series

# Save the combined dataframe to a new Excel file in the target folder
output_file = os.path.join(destination_folder, f'Total Power generation {year}.xlsx')
daily_time_series.to_excel(output_file, index=False)
